### To do

1. Make it easier to visualize jury trials/extract jury trial results and do the same for other variables
2. Make sure context + system prompt fits in context window
3. Add helper function to manually label
4. Add reranking algorithm

Reranking
- https://adasci.org/a-hands-on-guide-to-enhance-rag-with-re-ranking/
- https://techcommunity.microsoft.com/t5/microsoft-developer-community/doing-rag-vector-search-is-not-enough/ba-p/4161073
- https://community.openai.com/t/bad-formats-for-semantic-search-of-rag-implementing-internal-chatbot-for-troubleshooting-an-sdk/848715
- https://learn.microsoft.com/en-us/azure/search/index-similarity-and-scoring
- https://cohere.com/blog/rerank-3
- https://www.reddit.com/r/LocalLLaMA/comments/1d9h2pg/doing_rag_vector_search_is_not_enough/
- https://www.datacamp.com/tutorial/boost-llm-accuracy-retrieval-augmented-generation-rag-reranking
- https://python.langchain.com/v0.2/docs/integrations/retrievers/flashrank-reranker/

RAG
- https://ollama.com/blog/embedding-models
- https://huggingface.co/learn/nlp-course/chapter5/6
- https://docs.mistral.ai/guides/rag/
- https://docs.trychroma.com/guides

### Code

In [1]:
import numpy as np
import pandas as pd
import os
from utils.case_directory import CaseDirectory
from utils.case_metadata import CaseMetadata
from extractors.jury_ruling_classifier import JuryRulingClassifier

In [2]:
df = pd.read_csv("labeled_cases.csv")
df[df.trial_type == "jury"].metadata_path.tolist()

['workdata/100_random_sample/New_York_State_Suffolk_County_Supreme_Court/602235---2016/metadata.json',
 'workdata/100_random_sample/Delaware_District_Court/1--21-cv-01238/metadata.json',
 'workdata/100_random_sample/Massachusetts_State_Superior_Court_Essex_County/1777CV00789/metadata.json',
 'workdata/100_random_sample/Connecticut_State_Superior_Court/HHD-CV17-6080452-S/metadata.json',
 'workdata/100_random_sample/Connecticut_State_Superior_Court/UWY-CV22-6068059-S/metadata.json']

In [2]:
path = '100_random_sample/Connecticut_State_Superior_Court/HHD-CV17-6080452-S/metadata.json'
classifier = JuryRulingClassifier(path)

In [3]:
classifier.extract()

Extracting from metadata...
- Getting relevant chunks...


/Users/andrew/Downloads/summer_internship/Variable_Extraction/utils/case_metadata.py:139: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  lambda html: BeautifulSoup(html, features="html.parser").text
Number of requested results 8 is greater than number of elements in index 1, updating n_results = 1


- Querying llm...


KeyboardInterrupt: 